# Preprocessing

In [11]:
from gensim import corpora
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords

def tokenize(text, stopset):
    tokens = word_tokenize(text)
    return [w for w in tokens if not w in stopset]

df = pd.read_csv("lyrics.csv")
stopset = set(stopwords.words('english')).union(set(['chorus','verse','[',']','(',')']))

df = df[['index','song','artist','genre','lyrics']]
df = df[df.apply(lambda x: not (pd.isnull(x['lyrics']) or pd.isnull(x['song']) or x['genre'] == 'Not Available' or x['genre'] == 'Other'), 
        axis=1, reduce=True)]
df['lyrics'] = df.apply(lambda x: tokenize(x['lyrics'], stopset), axis=1, reduce=True)
df = df[df.apply(lambda x: len(x['lyrics']) > 20, axis=1, reduce=True)]
df = df.groupby('genre').apply(lambda x: x.sample(n=2000))

df.to_pickle('tickle_my.pkl')



# Word/Document Embedding

In [1]:
import pandas as pd
from gensim import corpora
from gensim.models import KeyedVectors

#create seeds
happy = ['happy','hope', 'joy', 'gratitude', 'satisfaction']
love = ['love','admiration', 'lust']
sadness = ['sadness','lonely', 'pity']
fear = ['fear','nervous', 'anxious', 'distress']
anger = ['anger','resentment', 'hate', 'disgust', 'frustration']

model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

#df = pd.read_pickle('tickle_my.pkl')
#lyrics = dataframe.as_matrix(['index'])


In [2]:
model.init_sims(replace=False)

In [8]:
model.most_similar(positive=anger,negative=love,topn=50)

[('discontent', 0.6435658931732178),
 ('unhappiness', 0.6304956078529358),
 ('dissatisfaction', 0.6197926998138428),
 ('disgruntlement', 0.6064539551734924),
 ('frustrations', 0.5987093448638916),
 ('displeasure', 0.597790539264679),
 ('outrage', 0.5904906988143921),
 ('discontentment', 0.5718801617622375),
 ('exasperation', 0.5645514726638794),
 ('disquiet', 0.557529091835022),
 ('indignation', 0.5535059571266174),
 ('hostility', 0.5502309799194336),
 ('fury', 0.5428645610809326),
 ('dismay', 0.5392096638679504),
 ('unease', 0.5373574495315552),
 ('disaffection', 0.5370421409606934),
 ('Frustration', 0.5344427227973938),
 ('disenchantment', 0.5272442102432251),
 ('animosity', 0.5169574022293091),
 ('bitterness', 0.5159363150596619),
 ('distrust', 0.513446569442749),
 ('impatience', 0.5115873217582703),
 ('irritation', 0.5097345113754272),
 ('hostility_toward', 0.5039492845535278),
 ('simmering_anger', 0.5033793449401855),
 ('disatisfaction', 0.5017848610877991),
 ('angst', 0.500400424